In [ ]:
import tensorflow as tf
import numpy as np
from numpy import asarray
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from keras .callbacks import LearningRateScheduler
from keras.wrappers.scikit_learn import KerasClassifier

from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers

import keras_tuner

from sklearn.utils import class_weight
import datetime 
import pathlib

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [ ]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
seed = 42
np.random.seed(42)

In [ ]:
def genAugmLayer(seed): 
    return tf.keras.Sequential([
     tfkl.RandomFlip("horizontal_and_vertical"),
     tfkl.RandomRotation(0.5),
     tfkl.RandomZoom( height_factor=(-0.3, 0.3), width_factor=(-0.3, 0.3),fill_mode = 'reflect',seed=seed) ])

In [ ]:
imbalClasses = tf.keras.utils.image_dataset_from_directory( 
    '/kaggle/input/selective-dataset/training_selective_data',
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=32,
    image_size=(96,96),
    subset = 'training',
    validation_split=0.15,
    seed=seed,
)

data_augmentation1 = genAugmLayer(17) 
dataAugmImbal = imbalClasses.map(lambda x , y : (data_augmentation1(x), y)) 
data_augmentation2 = genAugmLayer(271) 
dataAugmImbal = dataAugmImbal.concatenate(imbalClasses.map(lambda x , y : (data_augmentation2(x), y))) #double samples of classes 1 and 6 

In [ ]:
datapath = '/kaggle/input/training-data-final/training_data_final'
training_set = tf.keras.utils.image_dataset_from_directory(
    datapath,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=32, 
    image_size=(96,96),
    shuffle=True,
    seed=seed,
    validation_split=0.15,
    subset='training'
)

validation_set = tf.keras.utils.image_dataset_from_directory(
    datapath,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=32, 
    image_size=(96,96),
    shuffle=True,
    seed=seed,
    validation_split=0.15,
    subset='validation'
)


In [ ]:
data_augmentation = genAugmLayer(89)
training_set = training_set.concatenate(training_set.map(lambda x , y : (data_augmentation(x), y)) )
training_set = training_set.concatenate(dataAugmImbal)
training_set = training_set.concatenate(imbalClasses) 

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
training_set = training_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_set = validation_set.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
ds_labels_train = []
ds_images_train = []
for images, labels in training_set.unbatch():
    ds_labels_train.append(labels)
    ds_images_train.append(images)

ds_labels_val = []
ds_images_val = []
for images1, labels1 in validation_set.unbatch():
    ds_labels_val.append(labels1)
    ds_images_val.append(images1)
    
    

In [ ]:
from tensorflow.keras.utils import to_categorical

x_train = np.array(ds_images_train)
y_train = np.array(ds_labels_train)
x_val = np.array(ds_images_val)
y_val = np.array(ds_labels_val)

In [ ]:
input_shape = (96, 96, 3)

In [ ]:
supernet = tf.keras.applications.inception_v3.InceptionV3( include_top=False, weights='imagenet')
supernet.trainable = True

In [ ]:
training_set = training_set.map(lambda x,y: (tf.keras.applications.inception_v3.preprocess_input(x), y))
validation_set = validation_set.map(lambda x,y: (tf.keras.applications.inception_v3.preprocess_input(x),y))

In [ ]:
input_shape = (96, 96, 3)
def build_model(hp):
    inputs = tfkl.Input(input_shape)
    image_size = hp.Int("image_size", min_value=96, max_value=299, sampling="log")
    outputs = tfkl.Resizing(height = image_size, width = image_size, interpolation = 'bicubic' )(inputs)
    outputs = supernet(outputs)
    outputs = tfkl.GlobalAveragePooling2D()(outputs) 
    rate1 = hp.Float("drop_rate1", min_value=0.10, max_value=0.70, sampling="log")
    outputs = tfkl.Dropout(rate=rate1)(outputs)
    for i in range(hp.Int("num_layers", 1, 3)):
        outputs = layers.Dense(
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation=hp.Choice("activation", ["relu", "tanh"])
        )(outputs)
        
    rate2 = hp.Float("drop_rate2", min_value=0.10, max_value=0.70, sampling="log")
    outputs = tfkl.Dropout(rate=rate2)(outputs)

    outputs = layers.Dense(8, activation="softmax")(outputs)
    
    model = keras.Model(inputs, outputs)
    learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


build_model(keras_tuner.HyperParameters())

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="HypersTuning"
)


In [ ]:
tuner.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val))

In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/HypersTuning
Showing 10 best trials
Trial summary
Hyperparameters:
image_size: 181
drop_rate1: 0.36063659420260846
num_layers: 1
units: 224
activation: tanh
drop_rate2: 0.32604967508883836
lr: 0.0003243375670895756
Score: 0.9152542650699615
Trial summary
Hyperparameters:
image_size: 235
drop_rate1: 0.1288482296926094
num_layers: 2
units: 192
activation: relu
drop_rate2: 0.2243768124070131
lr: 8.151719073899244e-05
Score: 0.9105461537837982
Trial summary
Hyperparameters:
image_size: 285
drop_rate1: 0.31849876095258756
num_layers: 1
units: 224
activation: relu
drop_rate2: 0.19655137514741783
lr: 0.0006524426267004544
Score: 0.8898304998874664
Trial summary
Hyperparameters:
image_size: 137
drop_rate1: 0.6283380124412485
num_layers: 3
units: 256
activation: relu
drop_rate2: 0.11235492492081643
lr: 3.131455351631533e-05
Score: 0.8879472613334656
Trial summary
Hyperparameters:
image_size: 267
drop_rate1: 0.4573371602442971
num_layers: 2
units: 32
activation: